In [ ]:
# ======================================================
# 셀 1: 기본 설정 및 라이브러리 임포트
# ======================================================
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.losses import (binary_crossentropy)
from tensorflow.keras.mixed_precision import set_global_policy
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input

# 전역 정책을 'mixed_float16'으로 설정
set_global_policy('mixed_float16')

In [ ]:
# ======================================================
# 셀 2: 데이터셋 준비 및 모델 생성 함수 정의
# ======================================================

def prepare_datasets(base_path, seed, img_size, batch_size, train_ratio=0.7, val_ratio=0.15):
    """
    지정된 경로에서 전체 데이터셋을 로드하고 훈련, 검증, 테스트용으로 분할합니다.
    """
    print("전체 데이터셋 로딩 중...")
    full_dataset = tf.keras.utils.image_dataset_from_directory(
        base_path,
        seed=seed,
        image_size=img_size,
        batch_size=batch_size,
        shuffle=True
    )
    
    class_names = full_dataset.class_names
    print(f"총 {len(class_names)}개의 클래스를 찾았습니다.")

    dataset_size = tf.data.experimental.cardinality(full_dataset).numpy()
    train_size = int(train_ratio * dataset_size)
    validation_size = int(val_ratio * dataset_size)

    print(f"전체: {dataset_size} 배치 -> 훈련: {train_size}, 검증: {validation_size}, 테스트: {dataset_size - train_size - validation_size} 배치로 분할")

    train_ds = full_dataset.take(train_size)
    validation_ds = full_dataset.skip(train_size).take(validation_size)
    test_ds = full_dataset.skip(train_size + validation_size)

    AUTOTUNE = tf.data.AUTOTUNE
    train_ds = train_ds.shuffle(256).prefetch(buffer_size=AUTOTUNE)
    validation_ds = validation_ds.prefetch(buffer_size=AUTOTUNE)
    test_ds = test_ds.prefetch(buffer_size=AUTOTUNE)

    print("✅ 훈련, 검증, 테스트 데이터셋 분할 완료.")
    print("Found 150610 files belonging to 150 classes. ---- 이 문구와 다르다면 바로 말씀해주세요!!!!")
    return train_ds, validation_ds, test_ds, class_names

def create_kfood_model(input_shape, num_classes):
    base_model = tf.keras.applications.VGG16(
        include_top=False,
        weights='imagenet',
        input_shape=input_shape,
    )
    base_model.trainable = False

    inputs = tf.keras.Input(shape=input_shape)
    x = preprocess_input(inputs)
    x = base_model(x, training=False)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dropout(0.3)(x)
    outputs = tf.keras.layers.Dense(num_classes, activation="softmax", dtype='float32')(x)
    model = tf.keras.Model(inputs, outputs)


    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

In [ ]:
# ======================================================
# 셀 3: 메인 실행 - 데이터 준비, 모델 생성 및 1단계 학습
# ======================================================

seed = 53
tf.random.set_seed(seed)
np.random.seed(seed)

# 파라미터 설정
DATA_PATH = '../../데이터/data/kfood'
MODEL_PATH = '../model'
IMG_SIZE = (224, 224)
BATCH_SIZE = 64

# --- 데이터 준비 ---
train_dataset, validation_dataset, test_dataset, class_names = prepare_datasets(
    base_path=DATA_PATH,
    seed=seed,
    img_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

# --- 모델 생성 ---
model = create_kfood_model(
    input_shape=IMG_SIZE + (3,),
    num_classes=len(class_names)
)
model.summary()

# --- 1단계: 전이 학습 실행 ---
print("\n--- 1단계: 전이 학습 시작 ---")
es = tf.keras.callbacks.EarlyStopping(patience=5, monitor="val_loss", restore_best_weights=True, verbose=1)
ck_path = os.path.join(MODEL_PATH, "best_kfood_model_vgg.keras")
os.makedirs(MODEL_PATH, exist_ok=True)
ck = tf.keras.callbacks.ModelCheckpoint(ck_path, save_best_only=True, monitor='val_loss', verbose=1)

history_transfer = model.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=100,
    callbacks=[es, ck]
)

In [ ]:
# ======================================================
# 셀 4: 1단계 학습 결과 시각화
# ======================================================
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(history_transfer.history['loss'], label='Train Loss')
plt.plot(history_transfer.history['val_loss'], label='Validation Loss')
plt.title('Transfer Learning - Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(history_transfer.history['accuracy'], label='Train Accuracy')
plt.plot(history_transfer.history['val_accuracy'], label='Validation Accuracy')
plt.title('Transfer Learning - Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
# ======================================================
# 셀 5: 2단계 - 미세 조정 실행
# ======================================================
print("\n--- 2단계: 미세 조정 준비 ---")

# 가장 성능이 좋았던 모델을 로드
model = tf.keras.models.load_model(ck_path)

# 기반 모델 동결 해제 및 하위 레이어 재동결
base_model = model.layers[2]
base_model.trainable = True
for layer in base_model.layers[:-20]:
    layer.trainable = False

# 매우 낮은 학습률로 다시 컴파일
fine_tune_optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)
model.compile(
    optimizer=fine_tune_optimizer,
    loss='sparse_categorical_crossentropy',
    metrics=['sparse_categorical_accuracy']
)

print("\n미세 조정을 위한 모델 구조:")
model.summary()

# 미세 조정 학습 시작
print("\n--- 2단계: 미세 조정 시작 ---")
initial_epoch = history_transfer.epoch[-1] + 1
history_fine = model.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=20,
    initial_epoch=initial_epoch,
    callbacks=[es, ck] # 동일한 콜백 사용
)

In [ ]:
# ======================================================
# 셀 6: 최종 모델 평가
# ======================================================
print("\n--- 최종 성능 평가 ---")

# 최종적으로 가장 성능이 좋았던 모델 로드
best_model = tf.keras.models.load_model(ck_path)

# 학습에 전혀 사용되지 않은 테스트 데이터셋으로 최종 성능 평가
print("\nTest 데이터셋으로 최종 성능을 평가합니다...")
loss, accuracy = best_model.evaluate(test_dataset)
print(f"최종 테스트 손실 (Final Test Loss): {loss:.4f}")
print(f"최종 테스트 정확도 (Final Test Accuracy): {accuracy:.4f}")